In [46]:
# pirmas
import pandas as pd

data = [
    ("Vardas1", 0, 165, 55),
    ("Vardas2", 1, 183, 95),
    ("Vardas3", 0, 171, 60),
    ("Vardas4", 1, 194, 102),
    ("Vardas5", 1, "?", 80),
    ("Vardas6", 1, 185, 90),
]
# print(data)

df = pd.DataFrame(data, columns=['vardas', 'lytis', 'ugis', 'svoris'])
filteredTable = df.drop(df[df['ugis'] == "?"].index)
print(filteredTable)
print('---- \n')

print(f"all average {filteredTable['ugis'].mean()}")
print(f"all median {filteredTable['ugis'].median()}")
print('---- \n')

women = filteredTable[filteredTable['lytis'] == 0]
print(f"women average {women['ugis'].mean()}")
print(f"women median {women['ugis'].median()}")
print('---- \n')

men = filteredTable[filteredTable['lytis'] == 1]
print(f"men average {men['ugis'].mean()}")
print(f"men median {men['ugis'].median()}")
print('---- \n')

    vardas  lytis ugis  svoris
0  Vardas1      0  165      55
1  Vardas2      1  183      95
2  Vardas3      0  171      60
3  Vardas4      1  194     102
5  Vardas6      1  185      90
---- 

all average 179.6
all median 183.0
---- 

women average 168.0
women median 168.0
---- 

men average 187.33333333333334
men median 185.0
---- 

